In [5]:
import pandas as pd

### Carregar o arquivo para o dataframe

In [6]:

arquivo = "perfil_ATUAL.csv"
colunas= ['PERIODO','UF','MUNICIPIO','COD_MUNICIPIO_TSE','NR_ZONA','SEXO','FAIXA_ETARIA','GRAU_ESCOLAR','TOTAL']
data = pd.read_csv(arquivo, header= None, names=colunas)

df = pd.DataFrame(data)


### Bloco (n) return o n-esimo bloco

In [7]:
num_records_per_block = 5

In [8]:
def bloco(n=1):
    global num_records_per_block
    global arquivo
    global colunas
    skip = (int(n)-1)*num_records_per_block
    bloco = pd.read_csv(arquivo,skiprows = skip,header = None,  nrows= num_records_per_block,names = colunas)
    return bloco

In [9]:
bloco(6373)

,PERIODO,UF,MUNICIPIO,COD_MUNICIPIO_TSE,NR_ZONA,SEXO,FAIXA_ETARIA,GRAU_ESCOLAR,TOTAL
0,201806,BA,ALAGOINHAS,33138,163,FEMININO,25 A 34 ANOS,ENSINO M DIO COMPLETO,5569
1,201806,BA,ALAGOINHAS,33138,163,FEMININO,25 A 34 ANOS,ENSINO M DIO INCOMPLETO,1697
2,201806,BA,ALAGOINHAS,33138,163,FEMININO,25 A 34 ANOS,L E ESCREVE,196
3,201806,BA,ALAGOINHAS,33138,163,FEMININO,25 A 34 ANOS,SUPERIOR COMPLETO,1406
4,201806,BA,ALAGOINHAS,33138,163,FEMININO,25 A 34 ANOS,SUPERIOR INCOMPLETO,1251


In [10]:
num_records = df.shape[0]
num_blocks  = (num_records//num_records_per_block)+1
campo_chave = 'PERIODO'   #não é chave, só teste
campo_ord = 'PERIODO'     #só teste

### Faz uma lista com todos os blocos

desnecessário e ineficiente(melhor escrever em arquivos)
<p>melhor ainda n escrever de modo algum

In [11]:
blocos = [0]
def w_block(block_mod = 1):
    global blocos
   
    global num_records_per_block
    global df
    num_records = df.shape[0]
    num_blocks  = num_records//num_records_per_block
    for i in range(block_mod,num_blocks+1):
               
        blocos = blocos[:i-1] + [bloco(i)]
    return blocos


### Retorna o numero de blocos escritos abaixo

In [12]:
def num_w_block(block_mod = 1):
    
    global num_records_per_block
    global df
    num_records = df.shape[0]
    num_blocks  = (num_records//num_records_per_block)+1
    return (num_blocks - block_mod)        +1


### Procura por blocos ordenados

- Retorna o endereço: bloco e linha do bloco
<p>

 
- caso não encontre, endereço que deveria ocupar

bug: Em loop ao não encontrar

In [13]:
def search_on_block(bloco_n,campo,value,i=0):
    global campo_chave
    global num_records_per_block
    start = search_on_line_block(0+i,int(bloco_n),campo)
    end = search_on_line_block(num_records_per_block-(i+1),bloco_n,campo)
    if start == value:
        return bloco_n, 0+i
    if start < value:
        if end < value:
            return search_on_block(int(bloco_n - 1),campo,value)
        if end > value:
            return search_on_block(int(bloco_n),value,i+1,)
        if end == value:
            return bloco_n, num_records_per_block-i
    if start > value: 
        return search_on_block(int(bloco_n - 1),campo,value)
    

In [14]:
def search_on_block_heap(bloco_n,campo,value, start_line = 0):
    global num_records_per_block
    global num_blocks
    
    while bloco_n<= num_blocks:
        for line in range(num_records_per_block):
            if line >= start_line:
                record = search_on_line_block(line,int(bloco_n),campo)
                if record == value: 
                    return bloco_n, line
        bloco_n +=1
    return bloco_n-1

In [15]:
def search_on_line_block(line,bloco_n,campo):
    return bloco(bloco_n).loc[line,campo]


### Implementação select

In [16]:
def select_key_ord(campo, value ):
    global num_blocks
    global campo_chave
    global campo_ord
    
    if campo == campo_ord:
        bloco_n,line_block = search_on_block(num_blocks//2,campo,value)
        return bloco(int(bloco_n)).loc[line_block]
    
    
            
        

In [17]:
def select_key_heap(campo, value,start_block=1):
    global num_blocks
    global campo_chave
     
    if campo == campo_chave:
        #print (campo)
        bloco_n,line_block = search_on_block_heap(start_block,campo,value)
        return bloco(int(bloco_n)).loc[line_block]  


## Equi Joins

In [18]:
def equijoin(tb1,campo1,tb2,campo2):
    pass

In [19]:
def equi(camp1,camp2):
    b1,b2 = 32,33
    global num_records_per_block
    global num_blocks
    
    for l1 in range(num_records_per_block+1):
        for l2 in range(num_records_per_block+1):
            if search_on_line_block(l1,b1,camp1)==search_on_line_block(l2,b2,camp2):
                return [bloco(b1).loc[l1],bloco(b2).loc[l2]]

In [43]:
# compara um bloco da tabela1 com blocos da tabela2
# 
def equi_on_blockt1(bloco_n,campo1, campo2 = 'PERIODO', start_bloco2 = 0):
    global num_records_per_block
    global num_blocks
    line2 = 0
    conta_bloco = 1
    bloco_atual = start_bloco2
    
    for line1 in range(num_records_per_block):
        record1 = (search_on_line_block(line1,bloco_n,campo1))
        bloco_n,line2 = search_on_block_heap(start_bloco2,campo2,record1,line2)
        line2 += 1
        line2 = line2%5
        if bloco_atual != bloco_n: 
            bloco_atual = bloco_n
            conta_bloco += 1
        print (bloco_n, line2, conta_bloco)
        #print(record1)

In [40]:
def equi_table1(campo1, campo2 = 'PERIODO', start_bloco2 = 0):
    global num_blocks
    for bloco_n in range(num_blocks):
        equi_on_blockt1(bloco_n,campo1, campo2, start_bloco2)

In [41]:
def merge_sort_join():
    pass

In [42]:
equi_table1('PERIODO', 'PERIODO')

0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 

0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 0
0 0 0
0 1 0
0 2 0
0 3 0
0 4 

KeyboardInterrupt: 

### campo de testes

In [39]:
equi_on_blockt1(32,'PERIODO', 'PERIODO',23)

23 1 0
23 2 0
23 3 0
23 4 0
23 0 0


In [23]:
search_on_line_block(3,32,'PERIODO')

201806

In [25]:
select_key_heap('PERIODO',201806)

PERIODO                                     201806
UF                                              AC
MUNICIPIO                               ACREL NDIA
COD_MUNICIPIO_TSE                             1120
NR_ZONA                                          8
SEXO                                      FEMININO
FAIXA_ETARIA                               16 ANOS
GRAU_ESCOLAR         ENSINO FUNDAMENTAL INCOMPLETO
TOTAL                                            4
Name: 0, dtype: object

In [26]:
num_blocks

168318

In [27]:
equi('PERIODO','PERIODO')

[PERIODO                    201806
 UF                             AC
 MUNICIPIO            ASSIS BRASIL
 COD_MUNICIPIO_TSE            1570
 NR_ZONA                         6
 SEXO                     FEMININO
 FAIXA_ETARIA         25 A 34 ANOS
 GRAU_ESCOLAR           ANALFABETO
 TOTAL                          61
 Name: 0, dtype: object, PERIODO                    201806
 UF                             AC
 MUNICIPIO            ASSIS BRASIL
 COD_MUNICIPIO_TSE            1570
 NR_ZONA                         6
 SEXO                     FEMININO
 FAIXA_ETARIA         25 A 34 ANOS
 GRAU_ESCOLAR         L  E ESCREVE
 TOTAL                         118
 Name: 0, dtype: object]